In [1]:
import numpy as np
import os, cv2
import h5py
import tifffile
from tqdm import tqdm, trange
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from deepreg.predict import unwrapped_predict, normalize_batched_image
from deepreg.model.layer import Warping
import tensorflow as tf
import matplotlib.pyplot as plt


batch_size = 8
label_shape = (30, 3)
irregular = True

2025-05-28 00:34:58.344596: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-28 00:34:58.344677: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-28 00:34:58.346445: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Register a video in batches

In [2]:
#Tiff files for registration needs to start with the prefix RIG_
tiff_folder = "/mnt/jfban_input"
out_folder = tiff_folder

checkpoint_path = "/workspace/demo_notebook/test_run/save/ckpt-7"

max_frames = 2000
red_chan = 1

side_len = 1024
padding = np.array([[0,0],[0,0],[0,0],[1,1]])
z_depth = 3

log_dir = "/workspace/out_vids/logs"

files = os.listdir(tiff_folder)
print(files)
files = [f for f in files if (f.endswith(".tif") or f.endswith(".tiff")) and f.startswith("RIG_")] 
files = sorted(files, key = lambda x: int(x.split("_")[-1].split("-")[0]))
print(files)

['RIG_0-1999.tif']
['RIG_0-1999.tif']


In [3]:
inputs = tifffile.imread(os.path.join(tiff_folder, files[0]))

In [4]:
inputs.shape

(2000, 2, 1200, 1200)

In [5]:
if(len(inputs.shape) == 3):
    irregular = False
    inputs = inputs[:, np.newaxis, ...]

In [6]:
crop_shape = np.array((side_len, side_len))

crop_offset = (inputs.shape[2:] - crop_shape) / 2
assert np.all(crop_offset == crop_offset.astype(int))
crop_offset = crop_offset.astype(int)

## Crop the images
inputs_crop = inputs[:, :, crop_offset[0]:crop_offset[0] + crop_shape[0], crop_offset[1]:crop_offset[1] + crop_shape[1]]
assert np.all(inputs_crop.shape[2:] == crop_shape)

In [7]:
if irregular:
    input_red = inputs_crop[:, 1, ...]
else:
    input_red = inputs_crop[:, 0, ...]

batched_fixed_image = np.repeat(np.expand_dims(input_red[0], axis=0), batch_size, axis=0).astype(np.float32)

In [8]:
os.chdir("/workspace/scripts")
from register import set_GPU, register
# set_GPU(1)

In [9]:
# config_path = os.path.join(checkpoint_path.split("/save")[0], "config.yaml")
config_path = "/workspace/demo_notebook/test_run/config.yaml"
model = None

red_offset = 0

other_chan = (red_chan - 1) % inputs.shape[1]

batched_fixed_image_pad = np.pad(batched_fixed_image[..., np.newaxis], padding, "constant", constant_values=np.min(batched_fixed_image))

warping = Warping(fixed_image_size=(side_len, side_len, z_depth), batch_size=batch_size)

for file in files:
    inputs = tifffile.imread(os.path.join(tiff_folder, file))
    inputs_crop = inputs[:, :, crop_offset[0]:crop_offset[0] + crop_shape[0], crop_offset[1]:crop_offset[1] + crop_shape[1]]
    input_red = inputs_crop[:, red_chan, ...]
    input_green = inputs_crop[:, other_chan, ...]
    outs = np.zeros_like(inputs_crop)
    for frame in trange(0, max_frames, batch_size):
        if frame + batch_size > (max_frames):
            frame = (max_frames) - batch_size

        batched_moving_image = input_red[(frame - red_offset):(frame-red_offset)+batch_size].astype(np.float32)
        # batched_moving_image = np.log2((batched_moving_image - np.min(batched_moving_image)) + 1, dtype=np.float32)
        # batched_moving_image = np.pad(batched_moving_image[..., np.newaxis], padding, "constant", constant_values=0)
        batched_moving_image = np.pad(batched_moving_image[..., np.newaxis], padding, "constant", constant_values=np.min(batched_moving_image))
        # batched_moving_image = np.repeat(batched_moving_image[..., np.newaxis], z_depth, axis=-1)
        
        ddf_output, pred_fixed_image, model = unwrapped_predict(
            batched_fixed_image_pad,
            batched_moving_image,
            log_dir,
            label_shape,
            label_shape,
            model=model,
            model_ckpt_path=checkpoint_path,
            model_config_path=config_path,
        )


        gr_in = np.pad(input_green[(frame):(frame)+batch_size, ..., np.newaxis], padding, "constant", constant_values=0).astype(np.float32)
        outs[frame:(frame) + batch_size, other_chan] = warping(inputs=[ddf_output, gr_in]).numpy()[..., 1]
        
        batched_moving_image = input_red[(frame - red_offset):(frame-red_offset)+batch_size].astype(np.float32)
        batched_moving_image = np.pad(batched_moving_image[..., np.newaxis], padding, "constant", constant_values=0)
        outs[frame:(frame) + batch_size, red_chan] = warping(inputs=[ddf_output, batched_moving_image]).numpy()[..., 1]
        
        # outs[frame:(frame) + batch_size] = pred_fixed_image[..., 1] * 100
    # tifffile.imwrite(os.path.join(out_folder, "RED_WARPED_" + file), outs)

    tifffile.imwrite(os.path.join(out_folder, "WARPED-BOTH_" + file), outs, imagej=True, bigtiff=True)
    # break


  0%|          | 0/250 [00:00<?, ?it/s]

2025-05-28 00:37:42 | WARNING  | Log directory /workspace/out_vids/logs exists already.
2025-05-28 00:37:42 | WARNING  | Using customized configuration. The code might break if the config doesn't match the saved model.
Built inputs.
Built control points.
Concatenated images.
{'extract_levels': ListWrapper([0, 1, 2, 3]), 'name': 'local', 'num_channel_initial': 16}
Built backbone.
Built DDF.
Built warping.


  0%|          | 0/250 [00:02<?, ?it/s]


ValueError: moving and fixed images in paired loader are not identical
difference are [(('/workspace/test_h5_files', '10078to15011'), ('/workspace/test_h5_files', '15011to15011')), (('/workspace/test_h5_files', '10411to15011'), None), (('/workspace/test_h5_files', '10412to15011'), None), (('/workspace/test_h5_files', '10419to15011'), None), (('/workspace/test_h5_files', '10423to15011'), None), (('/workspace/test_h5_files', '10456to15011'), None), (('/workspace/test_h5_files', '10459to15011'), None), (('/workspace/test_h5_files', '10523to15011'), None), (('/workspace/test_h5_files', '12233to15011'), None), (('/workspace/test_h5_files', '13626to15011'), None), (('/workspace/test_h5_files', '13627to15011'), None), (('/workspace/test_h5_files', '13671to15011'), None), (('/workspace/test_h5_files', '13847to15011'), None), (('/workspace/test_h5_files', '14344to15011'), None), (('/workspace/test_h5_files', '14745to15011'), None), (('/workspace/test_h5_files', '15004to15011'), None), (('/works